In [ ]:
# get cpg sites idx
bedtools intersect -a ~/bed_hg38/bed_hg38.bed -b ~/celfie_work1/median_15/tims_median_15.bed -wa > ~/cfDNA_benchmark/exchange_marker/celfie/output.bed

In [ ]:
# get cpg sites idx
!awk -F'\t' '{print $NF}' output.bed > cpg_idx.csv
!echo "cpg_idx" > header.csv
!cat header.csv cpg_idx.csv > celfeer_idx.csv
!rm header.csv cpg_idx.csv

In [ ]:
# Generate ref based on bed-matched methylation ratios
import numpy as np
import pandas as pd
import os
import re

# Define median_value and median_range
median_range = [15]

def extract_number(col_name):
    match = re.search(r'_(\d+)_', col_name)
    if match:
        return int(match.group(1))
    return float('inf')  

out_path= '~/exchange_marker/celfie'
if not os.path.exists(out_path):
    os.makedirs(out_path)

for median_value in median_range:
    ref_file = f'~/exchange_marker/celfie/celfie_idx.csv'  
    ref_df = pd.read_csv(ref_file)

    result_df = pd.DataFrame()
    mixture_folder = '~/benchmark_pat/pat_merged/ref/' 
    beta_files = os.listdir(mixture_folder)
    beta_files = sorted(beta_files) 
    cpg_idx_values = ref_df['cpg_idx']
    new_df = pd.DataFrame({'cpg_idx': cpg_idx_values})

    for beta_file in beta_files:
        if beta_file.endswith('.beta'):
            beta_path = os.path.join(mixture_folder, beta_file)
            beta_content = np.fromfile(beta_path, dtype=np.uint8).reshape((-1, 2))

            cpg_idx = new_df['cpg_idx'] - 1

            meth = beta_content[cpg_idx, 0]
            covered = beta_content[cpg_idx, 1]

            meth_ratio = meth / covered

            column_name = os.path.splitext(beta_file)[0]
         
            result_df[column_name] = meth_ratio

    output_df = pd.concat([new_df, result_df], axis=1)
    output_file = f'{out_path}/celfie_{median_value}.csv'  
    
    output_df.to_csv(output_file, index=False)
    del ref_df, result_df, new_df


In [ ]:
# Generate mix based on bed-matched methylation ratios
import numpy as np
import pandas as pd
import os
import re

# Define median_value and median_range
METHODS = ['dirichlet_dis', 'cfsort_dis', 'uniform_dis', 'dirichlet']

def extract_number(col_name):
    match = re.search(r'_(\d+)_', col_name)
    if match:
        return int(match.group(1))
    return float('inf')  

out_path= '~/exchange_marker/celfie'
if not os.path.exists(out_path):
    os.makedirs(out_path)

for median_value in METHODS:
   
    ref_file = f'~/exchange_marker/celfie/celfie_idx.csv'  
    ref_df = pd.read_csv(ref_file)

    result_df = pd.DataFrame()

    mixture_folder = f'~/benchmark_pat/pat_merged/{median_value}/' 
    beta_files = os.listdir(mixture_folder)
    beta_files = sorted(beta_files) 

    cpg_idx_values = ref_df['cpg_idx']
    new_df = pd.DataFrame({'cpg_idx': cpg_idx_values})

    for beta_file in beta_files:
        if beta_file.endswith('.beta'):
            beta_path = os.path.join(mixture_folder, beta_file)
            beta_content = np.fromfile(beta_path, dtype=np.uint8).reshape((-1, 2))

            cpg_idx = new_df['cpg_idx'] - 1

            meth = beta_content[cpg_idx, 0]
            covered = beta_content[cpg_idx, 1]

            meth_ratio = meth / covered

            column_name = os.path.splitext(beta_file)[0]
                
            result_df[column_name] = meth_ratio

    output_df = pd.concat([new_df, result_df], axis=1)
    output_file = f'{out_path}/mix_{median_value}.csv'  
    
    columns_to_sort = output_df.columns[1:]

    sorted_columns = sorted(columns_to_sort, key=extract_number)

    output_df = output_df[[output_df.columns[0]] + sorted_columns]
    
    output_df.to_csv(output_file, index=False)
    del ref_df, result_df, new_df


In [ ]:
# run deconv
import subprocess

DISTRIBUTION = ['dirichlet_dis', 'cfsort_dis', 'uniform_dis', 'dirichlet']
out_path = '~/exchange_marker/celfie'

for median in DISTRIBUTION:
    command = [
        "python",
        "/home/sty/meth_atlas/deconvolve.py",
        "-a",
        f"{out_path}/celfie_15.csv",
        f"{out_path}/mix_{median}.csv",
        "-o", out_path
    ]
    
    # subprocess.run
    result = subprocess.run(command, capture_output=True, text=True)
    
    print(f"Command: {' '.join(command)}")
    print("Output:")
    print(result.stdout)
    print("Error:")
    print(result.stderr)

In [ ]:
# cal Eval Metrix